In [2]:
import pandas as pd
import numpy as np

# Read the course_master.csv file
df = pd.read_csv("course_master.csv")
n = len(df)

# Randomly shuffle the indices for later assignment of exam durations
indices = df.index.to_list()
np.random.shuffle(indices)

# Calculate the counts for each exam duration
count_180 = int(0.7 * n)
count_120 = int(0.2 * n)
count_90  = n - count_180 - count_120

# Assign exam duration values based on the computed counts
df.loc[indices[:count_180], "Exam Duration"] = 180
df.loc[indices[count_180:count_180+count_120], "Exam Duration"] = 120
df.loc[indices[count_180+count_120:], "Exam Duration"] = 90

# Define the days and times lists
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat']
times = ['08:00', '08:15', '08:30', '08:45',
         '09:00', '09:15', '09:30', '09:45',
         '10:00', '10:15', '10:30', '10:45',
         '11:00', '11:15', '11:30', '11:45',
         '12:00', '12:15', '12:30', '12:45', '13:00',
         '14:00', '14:15', '14:30', '14:45',
         '15:00', '15:15', '15:30', '15:45',
         '16:00', '16:15', '16:30', '16:45',
         '17:00', '17:15', '17:30', '17:45', '18:00']

lunch_time_index = times.index('13:00')
pre_lunch_times = list(range(lunch_time_index + 1))
post_lunch_times = list(range((lunch_time_index + 1), len(times)))

def begin_end_pairs(session_len:int) -> list:
    pairs = []
    for s in range(0, len(times) - session_len):
        e = s + session_len
        if (((s in pre_lunch_times) and (e in pre_lunch_times)) or
            ((s in post_lunch_times) and (e in post_lunch_times))):
            pairs.append((s, e))
    return pairs

# Create a helper function to convert time string to minutes since midnight.
def time_to_minutes(time_str):
    hh, mm = map(int, time_str.split(":"))
    return hh * 60 + mm

# Now build the "Preferred Timings" column.
# For each course, determine session_len from its exam duration (assuming durations are in multiples of 15),
# then get the candidate begin-end pairs and combine each with each day.
# Finally, randomly sample a few candidate triplets (here sample_size=3 per course).

sample_size = 3
preferred_timings = []

for idx, row in df.iterrows():
    duration = int(row["Exam Duration"])  # duration in minutes
    session_len = duration // 15           # session length in terms of time slots (15-min intervals)
    pairs = begin_end_pairs(session_len)
    
    candidate_triplets = []
    # Form candidate triplets: convert slot indices to actual time strings and combine with a day
    for s, e in pairs:
        begin_time = times[s]
        end_time = times[e]
        # Check that the difference equals the duration (for sanity)
        if time_to_minutes(end_time) - time_to_minutes(begin_time) == duration:
            for day in days:
                candidate_triplets.append((begin_time, end_time, day))
                
    # Randomly sample sample_size triplets (or fewer if not enough candidates)
    if candidate_triplets:
        indices_sample = np.random.choice(len(candidate_triplets), size=min(sample_size, len(candidate_triplets)), replace=False)
        sampled_triplets = [candidate_triplets[i] for i in indices_sample]
    else:
        sampled_triplets = []
    
    preferred_timings.append(sampled_triplets)

df["Preferred Timings"] = preferred_timings

# Save the updated dataframe back to course_master.csv
df.to_csv("course_master.csv", index=False)

In [1]:
import pandas as pd
import numpy as np

# Read the course_master.csv file
df = pd.read_csv("course_master.csv")
n = len(df)

# Keep exam duration assignments as before. (They can be random.)
indices = df.index.to_list()
np.random.shuffle(indices)
count_180 = int(0.7 * n)
count_120 = int(0.2 * n)
count_90  = n - count_180 - count_120
df.loc[indices[:count_180], "Exam Duration"] = 180
df.loc[indices[count_180:count_180+count_120], "Exam Duration"] = 120
df.loc[indices[count_180+count_120:], "Exam Duration"] = 90

# Define the days and times lists
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat']
times = ['08:00', '08:15', '08:30', '08:45',
         '09:00', '09:15', '09:30', '09:45',
         '10:00', '10:15', '10:30', '10:45',
         '11:00', '11:15', '11:30', '11:45',
         '12:00', '12:15', '12:30', '12:45', '13:00',
         '14:00', '14:15', '14:30', '14:45',
         '15:00', '15:15', '15:30', '15:45',
         '16:00', '16:15', '16:30', '16:45',
         '17:00', '17:15', '17:30', '17:45', '18:00']

lunch_time_index = times.index('13:00')
pre_lunch_times = list(range(lunch_time_index + 1))
post_lunch_times = list(range((lunch_time_index + 1), len(times)))

def begin_end_pairs(session_len: int) -> list:
    pairs = []
    for s in range(0, len(times) - session_len):
        e = s + session_len
        if (((s in pre_lunch_times) and (e in pre_lunch_times)) or
            ((s in post_lunch_times) and (e in post_lunch_times))):
            pairs.append((s, e))
    return pairs

def time_to_minutes(time_str):
    hh, mm = map(int, time_str.split(":"))
    return hh * 60 + mm

# For each course, compute and print all candidate triplets.
for idx, row in df.iterrows():
    duration = int(row["Exam Duration"])          # duration in minutes
    session_len = duration // 15                    # number of 15-minute intervals needed
    pairs = begin_end_pairs(session_len)
    
    candidate_triplets = []
    for s, e in pairs:
        begin_time = times[s]
        end_time = times[e]
        # Sanity check: the difference must equal the course's exam duration.
        if time_to_minutes(end_time) - time_to_minutes(begin_time) == duration:
            for day in days:
                candidate_triplets.append((begin_time, end_time, day))
    
    print(f"Course {idx} with exam duration {duration} mins:")
    print(candidate_triplets)
    print("=" * 60)

Course 0 with exam duration 180 mins:
[('08:00', '11:00', 'Mon'), ('08:00', '11:00', 'Tue'), ('08:00', '11:00', 'Wed'), ('08:00', '11:00', 'Thu'), ('08:00', '11:00', 'Fri'), ('08:00', '11:00', 'Sat'), ('08:15', '11:15', 'Mon'), ('08:15', '11:15', 'Tue'), ('08:15', '11:15', 'Wed'), ('08:15', '11:15', 'Thu'), ('08:15', '11:15', 'Fri'), ('08:15', '11:15', 'Sat'), ('08:30', '11:30', 'Mon'), ('08:30', '11:30', 'Tue'), ('08:30', '11:30', 'Wed'), ('08:30', '11:30', 'Thu'), ('08:30', '11:30', 'Fri'), ('08:30', '11:30', 'Sat'), ('08:45', '11:45', 'Mon'), ('08:45', '11:45', 'Tue'), ('08:45', '11:45', 'Wed'), ('08:45', '11:45', 'Thu'), ('08:45', '11:45', 'Fri'), ('08:45', '11:45', 'Sat'), ('09:00', '12:00', 'Mon'), ('09:00', '12:00', 'Tue'), ('09:00', '12:00', 'Wed'), ('09:00', '12:00', 'Thu'), ('09:00', '12:00', 'Fri'), ('09:00', '12:00', 'Sat'), ('09:15', '12:15', 'Mon'), ('09:15', '12:15', 'Tue'), ('09:15', '12:15', 'Wed'), ('09:15', '12:15', 'Thu'), ('09:15', '12:15', 'Fri'), ('09:15', '12:15

In [6]:


#Course 0 with exam duration 180 mins:
# The following are the preferred starting times for the course:
# 9:00 on Mon, 9:00 on Tue, 9:00 on Wed .....
# 11:00 on Mon, 11:00 on Tue, 11:00 on Wed .....

pref_time = ['09:30','10:30','11:30','14:00','15:00','16:00','17:00']
pref_days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']

def func(begin_time,pref_time):
    for x in pref_time:
        b1=time_to_minutes(begin_time)
        p1=time_to_minutes(x)
        if abs(b1-p1)<=30:
            return True
        
    return False

#Generate the preferred timings for each course based on the exam duration
# loop over all the possibilities of candidate slots for each course, If the starting time of the slot is within 30mins of the preferred time and the day is in the preferred days, then add it to the preferred timings

preferred_timings = []
for idx, row in df.iterrows():
    duration = int(row["Exam Duration"])          # duration in minutes
    session_len = duration // 15                    # number of 15-minute intervals needed
    pairs = begin_end_pairs(session_len)
    
    candidate_triplets = []
    for s, e in pairs:
        begin_time = times[s]
        end_time = times[e]
        # Sanity check: the difference must equal the course's exam duration.
        if time_to_minutes(end_time) - time_to_minutes(begin_time) == duration:
            for day in days:
                if func(begin_time,pref_time) and day in pref_days:
                    candidate_triplets.append((begin_time, end_time, day))
    
    preferred_timings.append(candidate_triplets)
    print(f"Course {idx} with exam duration {duration} mins:")
    print(candidate_triplets)
    print("=" * 60)

Course 0 with exam duration 180 mins:
[('09:00', '12:00', 'Mon'), ('09:00', '12:00', 'Tue'), ('09:00', '12:00', 'Wed'), ('09:00', '12:00', 'Thu'), ('09:00', '12:00', 'Fri'), ('09:15', '12:15', 'Mon'), ('09:15', '12:15', 'Tue'), ('09:15', '12:15', 'Wed'), ('09:15', '12:15', 'Thu'), ('09:15', '12:15', 'Fri'), ('09:30', '12:30', 'Mon'), ('09:30', '12:30', 'Tue'), ('09:30', '12:30', 'Wed'), ('09:30', '12:30', 'Thu'), ('09:30', '12:30', 'Fri'), ('09:45', '12:45', 'Mon'), ('09:45', '12:45', 'Tue'), ('09:45', '12:45', 'Wed'), ('09:45', '12:45', 'Thu'), ('09:45', '12:45', 'Fri'), ('10:00', '13:00', 'Mon'), ('10:00', '13:00', 'Tue'), ('10:00', '13:00', 'Wed'), ('10:00', '13:00', 'Thu'), ('10:00', '13:00', 'Fri'), ('14:00', '17:00', 'Mon'), ('14:00', '17:00', 'Tue'), ('14:00', '17:00', 'Wed'), ('14:00', '17:00', 'Thu'), ('14:00', '17:00', 'Fri'), ('14:15', '17:15', 'Mon'), ('14:15', '17:15', 'Tue'), ('14:15', '17:15', 'Wed'), ('14:15', '17:15', 'Thu'), ('14:15', '17:15', 'Fri'), ('14:30', '17:30